In [1]:
# 모듈 import

import os
import numpy as np
import random
import sys
sys.path.append("../mmdetection/")
# 이건 본인 환경의 mmdetection 폴더 잡아주면 됨

from mmengine.hooks import Hook
from mmengine.config import Config
from mmengine.runner import Runner
from mmdet.registry import DATASETS
from mmdet.utils import register_all_modules


from torch.utils.data import SubsetRandomSampler

In [ ]:
import wandb
wandb.login(key='2a631ea744b03506a1330798e0724d1d917a821f')

In [3]:
# 모든 모듈 등록
register_all_modules()

In [4]:
# config file 경로
cfg = Config.fromfile('../mmdetection/configs/faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py')

# 데이터셋 경로
root='./dataset/'

# work_dir 경로 (log랑 체크포인트 저장할 곳)
root_work_dir='./work_dirs/faster_rcnn_r50_fpn_1x_wandb_og'

# train max epochs 수
train_epochs = 12

# 체크포인트 최대 몇 개 저장해놓을 건지
keep_ckpts = 3

# 체크포인트 인터벌 (체크포인트 몇번째마다 저장할 건지)
ckpts_interval = 1


# -------------wandb--------------------------

# wandb 프로젝트 이름 (안 바꾸는 걸 추천함, 개인적으로 자기 것만 모아놓고 싶으면 바꿔도 됨)
wandb_project_name = 'Project2'


# wandb 실험 이름 (이건 자기가 원하는 걸로 변경해야 알아보기 편하겠지요?)
wandb_experiment_name = '원하는 이름'


# 몇 iter(False로 한 경우)마다 로그를 저장할 건지
# wandb에 로깅하는 것 뿐만 아니라 log에도 이 인터벌만큼만 찍히므로 참고해서 입력할 것
# 0은 못 받음, 0보다 큰 수로 줘야 함
# 숫자를 작게 줄수록 로깅을 자주 함(돌려보고 정하면 됨)
log_interval = 50


# 모델에 따라 아래 쉘의 num_classes 지정하는 부분이나 몇 부분 더 확인해볼 것 (roi_head 써야 하는지 bbox_head에 바로 줘야 하는지 같은 거)

In [ ]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")


# dataset config 수정
cfg.dataset_type = 'CocoDataset'
cfg.data_root = root

# Train 데이터셋 설정
train_dataset_cfg = dict(
    data_root=cfg.data_root,
    ann_file='train.json',
    data_prefix=dict(img=''),
    filter_cfg=dict(filter_empty_gt=True, min_size=32),
    pipeline=cfg.train_pipeline,
    metainfo=dict(classes=classes)
)

Dataset = DATASETS.get(cfg.dataset_type)
full_train_dataset = Dataset(**train_dataset_cfg)

# Train 데이터셋의 10%만 사용하기 위한 인덱스 선택
total_train_size = len(full_train_dataset)
subset_train_size = int(total_train_size * 0.1)
train_indices = random.sample(range(total_train_size), subset_train_size)

# 선택된 인덱스만 사용하는 새로운 Train 데이터셋 설정
train_dataset_cfg['indices'] = train_indices

# Train dataset config 수정
cfg.train_dataloader = dict(
    batch_size=4,
    num_workers=2,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    batch_sampler=dict(type='AspectRatioBatchSampler'),
    dataset=dict(
        type=cfg.dataset_type,
        **train_dataset_cfg
    )
)

# Test(Val) 데이터셋 설정
test_dataset_cfg = dict(
    data_root=cfg.data_root,
    ann_file='test.json',
    data_prefix=dict(img=''),
    test_mode=True,
    pipeline=cfg.test_pipeline,
    metainfo=dict(classes=classes)
)

full_test_dataset = Dataset(**test_dataset_cfg)

# Test 데이터셋의 10%만 사용하기 위한 인덱스 선택
total_test_size = len(full_test_dataset)
subset_test_size = int(total_test_size * 0.05)
test_indices = random.sample(range(total_test_size), subset_test_size)

# 선택된 인덱스만 사용하는 새로운 Test 데이터셋 설정
test_dataset_cfg['indices'] = test_indices

# Validation dataset config 수정 (Test와 동일하게 설정)
cfg.val_dataloader = dict(
    batch_size=1,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type=cfg.dataset_type,
        **test_dataset_cfg
    )
)

# Test dataset config 수정 (Validation과 동일하게 설정)
cfg.test_dataloader = cfg.val_dataloader

# Train, val, test evaluator 설정
cfg.train_evaluator = dict(
    type='CocoMetric',
    ann_file=cfg.data_root + 'train.json',
    metric='bbox',
    format_only=False
)

cfg.val_evaluator = dict(
    type='CocoMetric',
    ann_file=cfg.data_root + 'test.json',
    metric='bbox',
    format_only=False
)

cfg.test_evaluator = cfg.val_evaluator

# 기타 설정
cfg.train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=train_epochs, val_interval=1)
cfg.val_cfg = dict(type='ValLoop')
cfg.test_cfg = dict(type='TestLoop')

cfg.env_cfg = dict(
    cudnn_benchmark=False,
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0),
    dist_cfg=dict(backend='nccl'),
)

cfg.work_dir = root_work_dir


# 모델에 따라 선택하거나 아예 바꿔야 할 수도 있음
cfg.model.roi_head.bbox_head.num_classes = 10
#cfg.model.bbox_head.num_classes = 10


cfg.optim_wrapper.optimizer.lr = 0.02
cfg.optim_wrapper.clip_grad = dict(max_norm=35, norm_type=2)

In [6]:
# Hook, wandb, log, 시각화 관련 코드

# 체크포인트 pth와 로그 저장
cfg.default_hooks = dict(
    timer=dict(type='IterTimerHook'),
    param_scheduler=dict(type='ParamSchedulerHook'),
    checkpoint=dict(type='CheckpointHook', interval=ckpts_interval, max_keep_ckpts=keep_ckpts),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    visualization=dict(type='DetVisualizationHook'),
    logger=dict(type='LoggerHook', 
                log_metric_by_epoch=False,
                interval=log_interval,
                )
)

wandb_kwargs = dict(
    project=wandb_project_name,
    name=wandb_experiment_name,
)

visualizer = dict(
    type='DetLocalVisualizer',
    vis_backends=[
        dict(type='LocalVisBackend'),
        dict(type='WandbVisBackend',
             init_kwargs=wandb_kwargs)
    ],
    name='visualizer'
)

cfg.visualizer = visualizer

In [ ]:
# Runner 생성 및 학습 시작
runner = Runner.from_cfg(cfg)

runner.train()